# Albumentations

In [ ]:
import albumentations as A
import cv2
import matplotlib.pyplot as plt

import glob
import os
from IPython.display import display, Image
import shutil
import os
from tqdm.notebook import tqdm
import pandas as pd
from pandas.errors import EmptyDataError
import random
import shutil
import numpy as np
from skimage.util import random_noise
from random import shuffle


## откуда копировать (изображения и файлы разметки)
images_dir = "./data/"
labels_dir = "./data/"


## куда копировать
to_dir = './ALB_temp/'
shutil.rmtree(to_dir, ignore_errors = True)
os.makedirs(to_dir)

# создание списока файлов изображений
ext = ".JPG"
ext2 = ".jpg"
image_files = [
    f for f in os.listdir(images_dir) if f.endswith(ext) or f.endswith(ext2)
]

# создаем датафрейм с путями к изображениям и разметки
df = pd.DataFrame(columns = ['image_train', 'label_train'])
df['image_train'] = image_files
df['label_train'] = df['image_train'].apply(lambda x: x.replace(ext, '.txt').replace(ext2, '.txt'))
df = df[~df['label_train'].str.contains('alb_')]
df = df.sample(frac = 1)

# обработка изображений
for idx in tqdm(df.index):
    image_path = images_dir + df.loc[idx, 'image_train']
    image  = cv2.imread(image_path)
    label_path = labels_dir + df.loc[idx, 'label_train']
        
    if os.path.isfile(label_path):
        
        label = pd.read_csv(label_path, names = ['cn', 'xn', 'yn', 'h', 'w'], sep = ' ')       
        
        class_labels = list(label['cn'].values)
        bboxes_yolo = list(label.drop(['cn'], axis = 1).values)
    
        width, height = image.shape[0], image.shape[1]
        
        for i in range(3):
            random.seed(i)
            
            # список трансформаций изображений
            sq = [
                [0, 1, 2], 
                [3, 4, 5],
                [6, 0, 1],
                [4, 5, 6],
                [0, 5, 4],
            ]
            choice = random.choice(sq)
            
            for transform_choice in choice:                
                          
                if transform_choice == 0:
                    transform = A.Compose(
                        [
                            A.HorizontalFlip(p=0.5),
                            A.VerticalFlip(p=0.5),
                            A.RandomBrightnessContrast(p=0.5),
                    
                            A.ShiftScaleRotate(
                                shift_limit = 0.0625, 
                                scale_limit = 0.2, 
                                rotate_limit = 45, 
                                p = 0.5),
                        ],
                        bbox_params = A.BboxParams(
                            format = 'yolo', 
                            label_fields = ['class_labels']
                        )                    
                    )
                elif transform_choice == 1:
                    transform = A.Compose(
                        [
                            A.HorizontalFlip(p=1),
                            A.VerticalFlip(p=0.5),
                            A.RandomBrightnessContrast(p=0.5),
                    
                            A.OpticalDistortion(
                                distort_limit=(-0.25, 0.25),
                                p = 1
                            ),
                        ],
                        bbox_params = A.BboxParams(
                            format = 'yolo', 
                            label_fields = ['class_labels'],
                        )                   
                    )
            
                elif transform_choice == 2:
                    transform = A.Compose(
                        [
                            A.HorizontalFlip(p=0.5),
                            A.VerticalFlip(p=1),
                            A.RandomBrightnessContrast(p=0.5),
                    
                            A.GridDistortion(
                                num_steps=6,
                                distort_limit=(-1, 1),
                                p = 1),
                        ],
                        bbox_params = A.BboxParams(
                            format = 'yolo', 
                            label_fields = ['class_labels'],
                        )
                    )
            
                elif transform_choice == 3:
                    transform = A.Compose(
                        [
                            A.HorizontalFlip(p=0.5),
                            A.VerticalFlip(p=0.5),
                            A.RandomBrightnessContrast(p=0.5),
                    
                            A.HueSaturationValue(
                                hue_shift_limit = 20, 
                                sat_shift_limit = 50, 
                                val_shift_limit = 50, 
                                p = 1),
                        ],                    
                        bbox_params = A.BboxParams(
                            format = 'yolo', 
                            label_fields = ['class_labels'],
                        )
                    )
            
                elif transform_choice == 4:
                    transform = A.Compose(
                        [
                            A.HorizontalFlip(p=0.5),
                            A.VerticalFlip(p=0.5),
                            A.RandomBrightnessContrast(p=0.5),
                    
                            A.Defocus(
                                radius=(4, 8), 
                                alias_blur=(0.2, 0.4), 
                                always_apply=True,
                                p=1),
                        ],                    
                        bbox_params = A.BboxParams(
                            format = 'yolo', 
                            label_fields = ['class_labels'],
                        )
                    )
                
                elif transform_choice == 5:
                        transform = A.Compose(
                            [
                                A.HorizontalFlip(p=0.5),
                                A.VerticalFlip(p=0.5),
                                A.RandomBrightnessContrast(p=0.5),
                    
                                A.RandomRain(
                                    brightness_coefficient=0.9, 
                                    drop_width=1, 
                                    blur_value=5, 
                                    p=1),
                            ],
                            bbox_params = A.BboxParams(
                                format = 'yolo', 
                                label_fields = ['class_labels'],
                            )
                        )
            
                elif transform_choice == 6:
                    transform = A.Compose(
                        [
                            A.HorizontalFlip(p=0.5),
                            A.VerticalFlip(p=0.5),
                            A.RandomBrightnessContrast(p=0.5),
                        
                            A.RandomRain(
                                brightness_coefficient=0.9, 
                                drop_width=3, 
                                blur_value=5, 
                                p=1),
                        ],
                        bbox_params = A.BboxParams(
                            format = 'yolo', 
                            label_fields = ['class_labels'],
                        )
                    )
                try:
                    transformed_instance = transform(
                        image = image, 
                        bboxes = bboxes_yolo, 
                        class_labels = class_labels
                    )
                    transformed_image = transformed_instance['image']
                    transformed_bboxes = transformed_instance['bboxes']
    
                    transformed_bboxes = pd.DataFrame(transformed_bboxes, columns = ['xn', 'yn', 'h', 'w'])
                    transformed_bboxes = np.round(transformed_bboxes, 8)
                    transformed_bboxes['cn'] = class_labels
                    transformed_bboxes = transformed_bboxes[['cn', 'xn', 'yn', 'h', 'w']]
            
                    if transformed_bboxes.shape[0] > 0:
                        transformed_bboxes.to_csv(
                            to_dir + 'alb_' + str(transform_choice) + '_' + df.loc[idx, 'label_train'], 
                            index = False, 
                            header = None,
                            sep = ' '
                        )
                        cv2.imwrite(
                            to_dir + 'alb_' + str(transform_choice) + '_' + df.loc[idx, 'image_train'], 
                            transformed_image
                        )
            
                except:
                    pass